In [37]:
import pandas as pd
import hvplot.pandas
import requests

from api_key import geoapify_key


In [38]:
dataset_path = "./dataset_immigrantstat.csv"
immigrant_stat_pd = pd.read_csv(dataset_path)
immigrant_stat_pd

,REF_DATE,GEO,DGUID,Age (15C),Gender (3),Statistics (2),Immigrant status and period of immigration (9),VALUE
0,2021,Canada,2021A000011124,Total - Age,Total - Gender,2021 Counts,Total - Immigrant status and period of immigra...,36328475
1,2021,Canada,2021A000011124,Total - Age,Total - Gender,2021 Counts,Non-immigrants,27042120
2,2021,Canada,2021A000011124,Total - Age,Total - Gender,2021 Counts,Immigrants,8361505
3,2021,Canada,2021A000011124,Total - Age,Total - Gender,2021 Counts,Before 2001,3976410
4,2021,Canada,2021A000011124,Total - Age,Total - Gender,2021 Counts,2001 to 2005,911240
...,...,...,...,...,...,...,...,...
1552,2021,Nunavut,2021A000262,Total - Age,Total - Gender,2021 Counts,2001 to 2005,170
1553,2021,Nunavut,2021A000262,Total - Age,Total - Gender,2021 Counts,2006 to 2010,230
1554,2021,Nunavut,2021A000262,Total - Age,Total - Gender,2021 Counts,2011 to 2015,205
1555,2021,Nunavut,2021A000262,Total - Age,Total - Gender,2021 Counts,2016 to 2021,240


In [39]:
immigrant_stat_pd1 = immigrant_stat_pd[immigrant_stat_pd['GEO'].str.endswith(', Ont.')]
immigrant_stat_pd1

,REF_DATE,GEO,DGUID,Age (15C),Gender (3),Statistics (2),Immigrant status and period of immigration (9),VALUE
499,2021,"Barrie (CMA), Ont.",2021S0503568,Total - Age,Total - Gender,2021 Counts,Total - Immigrant status and period of immigra...,210155
500,2021,"Barrie (CMA), Ont.",2021S0503568,Total - Age,Total - Gender,2021 Counts,Non-immigrants,171400
501,2021,"Barrie (CMA), Ont.",2021S0503568,Total - Age,Total - Gender,2021 Counts,Immigrants,35340
502,2021,"Barrie (CMA), Ont.",2021S0503568,Total - Age,Total - Gender,2021 Counts,Before 2001,20395
503,2021,"Barrie (CMA), Ont.",2021S0503568,Total - Age,Total - Gender,2021 Counts,2001 to 2005,3465
...,...,...,...,...,...,...,...,...
913,2021,"Woodstock (CA), Ont.",2021S0504544,Total - Age,Total - Gender,2021 Counts,2001 to 2005,550
914,2021,"Woodstock (CA), Ont.",2021S0504544,Total - Age,Total - Gender,2021 Counts,2006 to 2010,800
915,2021,"Woodstock (CA), Ont.",2021S0504544,Total - Age,Total - Gender,2021 Counts,2011 to 2015,835
916,2021,"Woodstock (CA), Ont.",2021S0504544,Total - Age,Total - Gender,2021 Counts,2016 to 2021,950


In [75]:
clean_df = pd.DataFrame(columns=[
    "Name",
    "lat",
    "lng",
    "Non-immigrants",
    "Immigrants",
    "Before 2001",
    "2001 to 2005",
    "2006 to 2010",
    "2011 to 2015",
    "2016 to 2021",
    "Non-permanent residents"])
for index, row in immigrant_stat_pd1.iterrows():
    region_name = row["GEO"]
    clean_df.at[region_name, "Name"] = row["GEO"].replace("(CMA)", "").replace("(CA)", "").replace("Ont.", "Ontario")
    if row["Immigrant status and period of immigration (9)"] == "Non-immigrants":
        clean_df.at[region_name, "Non-immigrants"] = int(row["VALUE"])
    elif row["Immigrant status and period of immigration (9)"] == "Immigrants":
        clean_df.at[region_name, "Immigrants"] = int(row["VALUE"])
    elif row["Immigrant status and period of immigration (9)"] == "Before 2001":
        clean_df.at[region_name, "Before 2001"] = int(row["VALUE"])
    elif row["Immigrant status and period of immigration (9)"] == "2001 to 2005":
        clean_df.at[region_name, "2001 to 2005"] = int(row["VALUE"])
    elif row["Immigrant status and period of immigration (9)"] == "2006 to 2010":
        clean_df.at[region_name, "2006 to 2010"] = int(row["VALUE"])
    elif row["Immigrant status and period of immigration (9)"] == "2011 to 2015":
        clean_df.at[region_name, "2011 to 2015"] = int(row["VALUE"])
    elif row["Immigrant status and period of immigration (9)"] == "2016 to 2021":
        clean_df.at[region_name, "2016 to 2021"] = int(row["VALUE"])
    elif row["Immigrant status and period of immigration (9)"] == "Non-permanent residents":
        clean_df.at[region_name, "Non-permanent residents"] = int(row["VALUE"])
clean_df = clean_df.reset_index()
clean_df = clean_df.drop("index", axis=1)
clean_df

,Name,lat,lng,Non-immigrants,Immigrants,Before 2001,2001 to 2005,2006 to 2010,2011 to 2015,2016 to 2021,Non-permanent residents
0,"Barrie , Ontario",NaN,NaN,171400,35340,20395,3465,3545,3780,4160,3405
1,"Belleville - Quinte West , Ontario",NaN,NaN,99070,9065,6040,730,740,685,870,1115
2,"Brantford , Ontario",NaN,NaN,120245,20235,12485,1735,1855,1885,2265,1430
3,"Brockville , Ontario",NaN,NaN,27930,2855,2075,155,200,165,260,95
4,"Centre Wellington , Ontario",NaN,NaN,27085,3450,2615,215,225,175,220,110
5,"Chatham-Kent , Ontario",NaN,NaN,92315,9245,6290,830,690,585,845,530
6,"Cobourg , Ontario",NaN,NaN,17190,2550,2080,100,75,105,195,40
7,"Collingwood , Ontario",NaN,NaN,20945,3205,2340,215,215,160,285,120
8,"Cornwall , Ontario",NaN,NaN,54655,4525,2490,455,450,475,655,570
9,"Elliot Lake , Ontario",NaN,NaN,10030,1175,1090,35,15,15,25,15


In [91]:
# start finding lat and lng for each region
geocode_base_url = "https://api.geoapify.com/v1/geocode/search"
geocode_params = {
    "text":"",
    "lang":"en",
    "limit":1,
    "type":"city",
    "state": "ON",
    "filter":"countrycode:ca",
    "apiKey":geoapify_key, 
}
for index, row in clean_df.iterrows():
    print("Searching for ", row["Name"])
    geocode_params["text"] = row["Name"]
    resp = requests.get(geocode_base_url, params=geocode_params)
    if resp.status_code == 200:
        geocode_data = resp.json()
        if len(geocode_data["features"]) > 0:
            clean_df.at[index, "lat"] = geocode_data["features"][0]["properties"]["lat"]
            clean_df.at[index, "lng"] = geocode_data["features"][0]["properties"]["lon"]

clean_df

Searching for  Barrie , Ontario
Searching for  Belleville - Quinte West , Ontario
Searching for  Brantford , Ontario
Searching for  Brockville , Ontario
Searching for  Centre Wellington , Ontario
Searching for  Chatham-Kent , Ontario
Searching for  Cobourg , Ontario
Searching for  Collingwood , Ontario
Searching for  Cornwall , Ontario
Searching for  Elliot Lake , Ontario
Searching for  Essa , Ontario
Searching for  Greater Sudbury , Ontario
Searching for  Guelph , Ontario
Searching for  Hamilton , Ontario
Searching for  Hawkesbury (Ontario part) , Ontario
Searching for  Ingersoll , Ontario
Searching for  Kawartha Lakes , Ontario
Searching for  Kenora , Ontario
Searching for  Kingston , Ontario
Searching for  Kitchener - Cambridge - Waterloo , Ontario
Searching for  London , Ontario
Searching for  Midland , Ontario
Searching for  Norfolk , Ontario
Searching for  North Bay , Ontario
Searching for  Orillia , Ontario
Searching for  Oshawa , Ontario
Searching for  Ottawa - Gatineau (Ontari

,Name,lat,lng,Non-immigrants,Immigrants,Before 2001,2001 to 2005,2006 to 2010,2011 to 2015,2016 to 2021,Non-permanent residents
0,"Barrie , Ontario",44.389311,-79.690174,171400,35340,20395,3465,3545,3780,4160,3405
1,"Belleville - Quinte West , Ontario",44.098825,-77.574855,99070,9065,6040,730,740,685,870,1115
2,"Brantford , Ontario",43.140816,-80.263173,120245,20235,12485,1735,1855,1885,2265,1430
3,"Brockville , Ontario",44.589593,-75.684333,27930,2855,2075,155,200,165,260,95
4,"Centre Wellington , Ontario",43.718093,-80.374774,27085,3450,2615,215,225,175,220,110
5,"Chatham-Kent , Ontario",41.875000,-82.238725,92315,9245,6290,830,690,585,845,530
6,"Cobourg , Ontario",43.959600,-78.167778,17190,2550,2080,100,75,105,195,40
7,"Collingwood , Ontario",44.502723,-80.217238,20945,3205,2340,215,215,160,285,120
8,"Cornwall , Ontario",45.018442,-74.728703,54655,4525,2490,455,450,475,655,570
9,"Elliot Lake , Ontario",46.373464,-82.652893,10030,1175,1090,35,15,15,25,15


In [93]:
# %%capture --no-display

# put data on the map
clean_df = clean_df.astype({
    "lat": "float64",
    "lng": "float64",
    "Non-immigrants": "int64",
    "Immigrants": "int64",
    "Before 2001": "int64",
    "2001 to 2005": "int64",
    "2006 to 2010": "int64",
    "2011 to 2015": "int64",
    "2016 to 2021": "int64",
    "Non-permanent residents": "int64",
})
map_plot_1 = clean_df.hvplot.points(
    "lng",
    "lat",
    geo = True,
    tiles = "EsriNatGeo",
    hover_cols=["Name", "Non-immigrants", "Immigrants", "Non-permanent residents", "" ],
    size="Immigrants",
    scale=0.01,
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lng,lat]   (Immigrants,Name,Non-immigrants,Non-permanent residents)